In [20]:
import numpy as np

In [21]:
data = {
    'docs':[
        'my dog has flea problems help please',
        'maybe not take him to dog park stupid',
        'my falmation is so cute I love him',
        'stop posting stupid worthless garbage',
        'mr licks ate my steak how to stop him',
        'quit buying worthless dog food stupid',
    ],
    'labels':[0,1,0,1,0,1]
}

In [22]:
data['docs']  = list(map(lambda doc:doc.split(' '),data['docs']))

In [23]:
def getDocList(docs):
    # 初始化空集合
    docSet = set([])
    for doc in  docs:
        # 取交集
        docSet = set(doc) | docSet
    docList = list(docSet)
    docList.sort()
    return docList

In [24]:
docList = getDocList(data['docs'])

In [25]:
docList.index('steak')

26

In [26]:
def doc2V(doc,docList):
    # 单词集合 的 维度
    dims = len(docList)
    # 初始化 坐标点
    doc_v = [0]*dims

    for word in doc:
        if word in docList:
            doc_v[docList.index(word)] += 1
    # 文档 ——> 向量
    return doc_v
    

In [27]:
docs_v  = list(map(lambda doc:doc2V(doc,docList),data['docs']))

In [28]:
data['docs']

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'falmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [29]:
print(doc2V(data['docs'][0],docList))

[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [30]:
# P(A|B) * P(B) = P(B|A) * P(A)

In [31]:
data['labels']

[0, 1, 0, 1, 0, 1]

In [32]:
'''
函数功能：
    贝叶斯训练
输入：
    docs_v：docs 的 向量矩阵
    labels: 标签list
    docList：doc 集合 List（向量空间的基）
输出：
    P(Doc|C1), P(Doc|C0), P()
'''

def train(docs_v,labels,docList):
    # Abusive
    n_doc = len(labels)
    # 向量基 永远是 docList
    docLen = len(docList)

    # 防止零概率出现
    p1num = np.ones(docLen)
    p0num = np.ones(docLen)
    p1Denum,p0Denum = 2, 2
    # 遍历 所有的 doc 向量
    for i in range(n_doc):
        if labels[i] == 1:
            p1num += docs_v[i]
            p1Denum += np.sum(docs_v[i])
        elif labels[i] == 0:
            p0num += docs_v[i]
            p0Denum += np.sum(docs_v[i])
    # p1doc, p0doc, pA
    return np.log(p1num/p1Denum), np.log(p0num/p0Denum),np.sum(labels)/n_doc
    

In [33]:
p1doc,p0doc,pA = train(docs_v,data['labels'],docList)

In [34]:
p1doc

array([-3.04452244, -3.04452244, -2.35137526, -3.04452244, -1.94591015,
       -3.04452244, -3.04452244, -2.35137526, -2.35137526, -3.04452244,
       -3.04452244, -2.35137526, -3.04452244, -3.04452244, -3.04452244,
       -3.04452244, -2.35137526, -3.04452244, -3.04452244, -2.35137526,
       -2.35137526, -3.04452244, -2.35137526, -3.04452244, -2.35137526,
       -3.04452244, -3.04452244, -2.35137526, -1.65822808, -2.35137526,
       -2.35137526, -1.94591015])

In [35]:
'''
P(A|B) * P(B)  = P(B|A) * P(A)

如何计算 P(doc|C1)
P([word0,word1,word2,...,wordN]|C1) = 
P(word0|C1) * P(word1|C1) * ... * P(wordN|C1) => * 转换成 + 

sum(log([P(word0|C1)]) + log([P(word1|C1)]) + ... + log([P(wordN|C1)]))
'''

'\nP(A|B) * P(B)  = P(B|A) * P(A)\n\n如何计算 P(doc|C1)\nP([word0,word1,word2,...,wordN]|C1) = \nP(word0|C1) * P(word1|C1) * ... * P(wordN|C1) => * 转换成 + \n\nsum(log([P(word0|C1)]) + log([P(word1|C1)]) + ... + log([P(wordN|C1)]))\n'

In [36]:
'''
函数功能：
    贝叶斯训练：
输入：
    P1doc， p0doc, pA
输出：
    1/0

'''

def classify(doc,p1doc,p0doc,pA):
    doc_v  = doc2V(doc,docList)
    # P(doc|C1) = np.sum(np.log(doc_v * p1doc)) * P(C1)
    p1 = np.sum(doc_v * p1doc) + np.log(pA)
    p0 = np.sum(doc_v * p0doc) + np.log(1-pA)

    if p1 > p0:
        return 1
    else:
        return 0

In [37]:
docv  = doc2V('love my dalmation'.split(' '), docList)
docv * p1doc

array([-0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -3.04452244, -0.        , -0.        , -3.04452244, -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        ])

In [38]:
classify('love my dalmation'.split(' '),p1doc,p0doc,pA)
docv * p1doc

array([-0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -3.04452244, -0.        , -0.        , -3.04452244, -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        ])